<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Onehot_A" data-toc-modified-id="Onehot_A-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Onehot_A</a></span></li><li><span><a href="#导入与分割数据" data-toc-modified-id="导入与分割数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入与分割数据</a></span></li><li><span><a href="#A_cat" data-toc-modified-id="A_cat-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A_cat</a></span></li><li><span><a href="#A_hour" data-toc-modified-id="A_hour-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A_hour</a></span></li><li><span><a href="#A_xgb" data-toc-modified-id="A_xgb-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>A_xgb</a></span></li><li><span><a href="#B_cat" data-toc-modified-id="B_cat-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>B_cat</a></span></li><li><span><a href="#A_his" data-toc-modified-id="A_his-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A_his</a></span></li><li><span><a href="#结论" data-toc-modified-id="结论-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>结论</a></span></li><li><span><a href="#'C':-0.1" data-toc-modified-id="'C':-0.1-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>'C': 0.1</a></span></li></ul></div>

# Onehot_A

In [2]:
# coding: utf-8

# filelist: 
#         train:40428967,  
#     minitrain:4042898,  
# miniminitrain:404291,  
#    test_click:4577464
#  

#import os
#os.chdir('/media/zhou/0004DD1700005FE8/AI/00/project_2/')
#os.chdir('E:/AI/00/project_2')

class flags(object):
    def __init__(self, file_name, onehot_cat):
        self.file_name = file_name
        self.onehot_cat = onehot_cat
        self.data_dir = '../data/project_2/data/{0}/'.format(self.onehot_cat)
        self.output_dir = '../data/project_2/output/{0}/'.format(self.onehot_cat)
        self.model_dir = '../data/project_2/models/{0}/'.format(self.onehot_cat)

class params(object):
    def __init__(self, onehot_name):
        self.threshold = 10
        self.chunksize = 1e3
        self.num_trees = 50
        self.deep = 15
        self.split = '='
        # ['A_cat', 'A_hour', 'A_xgb', 
        #  'B_cat', 'C_his']
        self.onehot_name = onehot_name
        self.lr_C = 1


In [3]:
def MergeNpz(output_path, file_name, data_begin, threshold=10, is_click=False):
    """把生成的多个.npz合并成一个文件并保存"""
    #导入从0开始的文件, 为下面的合并做准备
    click_name = 'more'
    if is_click: 
        click_name = 'click'
        threshold = ''
    
    X_train = ss.load_npz(output_path+'{0}_X_{3}{1}_begin{2}.npz'.format(
                                file_name, threshold, data_begin[0],click_name))
    y_train = ss.load_npz(output_path+'{0}_y_{3}{1}_begin{2}.npz'.format(
                                file_name, threshold, data_begin[0],click_name))

    for begin in range(1,len(data_begin)): #循环读入文件
        #文件暂存
        X_train_tmp = ss.load_npz(output_path+'{0}_X_{3}{1}_begin{2}.npz'.format(
                                    file_name, threshold, data_begin[begin],click_name))
        y_train_tmp = ss.load_npz(output_path+'{0}_y_{3}{1}_begin{2}.npz'.format(
                                    file_name, threshold, data_begin[begin],click_name))
        X_train = ss.vstack((X_train, X_train_tmp))  #与原有 行稀疏矩阵 进行 行连接
        y_train = ss.hstack((y_train, y_train_tmp))  #与原有 列稀疏矩阵 进行 列连接
    
    print('total shape: ',X_train.shape, ' | ', y_train.shape)
    #保存最终连接完成的稀疏矩阵
    #ss.save_npz(output_path+'{0}_X_more{1}'.format(file_name, threshold), X_train)
    #ss.save_npz(output_path+'{0}_y_more{1}'.format(file_name, threshold), y_train)
    #print('saved in {0}'.format(output_path))
    return X_train, y_train


In [4]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
#import xgboost as xgb
#from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

In [4]:
totalsize = {'minitrain':4042898, 'test_click':4577464}
file_size = totalsize[FLAGS.file_name]  #总的数据量
block_size = 100000  #数据块大小

#定义参数
data_path = FLAGS.data_dir  # + '../data/project_2/'
file_name = FLAGS.file_name
chunksize = 1000
threshold = 10
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

data_begins = [i for i in range(0,file_size,block_size)]



In [ ]:
import line_profiler
import sys


def test():
    print 'haha'

prof = line_profiler.LineProfiler(MergeNpz)
prof.enable()  # 开始性能分析
a, b = MergeNpz(output_path, file_name, data_begins[:-6], threshold, False)
prof.disable()  # 停止性能分析
prof.print_stats(sys.stdout)

# 导入与分割数据

# A_cat

In [43]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [6]:
#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [18]:
X_train.shape, X_val.shape

((3500000, 646491), (542898, 646491))

In [8]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')


In [9]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached w

{'mean_fit_time': array([ 329.47616482,  369.60390115,  324.4078722 ,  367.42991328,
         212.2998058 ,  372.03892843,  336.70420321,  382.0876294 ,
         336.89638193,  373.32977041]),
 'mean_score_time': array([ 1.03666671,  1.0390203 ,  1.03444759,  1.04006028,  1.03586078,
         1.03757874,  1.04104495,  1.0464867 ,  1.04656116,  1.04050708]),
 'mean_test_score': array([-0.40504352, -0.40502292, -0.40257904, -0.40257576, -0.40604935,
        -0.40604334, -0.42796114, -0.42196957, -0.44536789, -0.4290202 ]),
 'mean_train_score': array([-0.39785209, -0.39785271, -0.38816782, -0.38816793, -0.36360968,
        -0.36365825, -0.3358255 , -0.33998795, -0.3302141 , -0.33608356]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga'],
              mask =

In [10]:
print(-grid.best_score_)
print(grid.best_params_)

0.402575757209
{'solver': 'saga', 'C': 0.1}


In [44]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.1

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 


/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


cost time:512
Train Accuary: 83.53%
Train log_loss:  0.388722417524


['../data/project_2/models/Onehot_A/LR_C0.1_A_cat.sklearn']

# A_hour

In [41]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_hour')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [11]:
#导入数据

print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [13]:
X_train.shape, y_train.shape

((4042898, 38), (4042898,))

In [14]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')


In [15]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached w

{'mean_fit_time': array([  33.70156749,  207.99539844,  183.90029987,  207.61317428,
         201.66621614,  207.55812836,  198.12250717,  208.25508253,
         107.69542424,   49.87002707]),
 'mean_score_time': array([ 0.54462361,  0.55181026,  0.54617755,  0.55055491,  0.55153791,
         0.55444105,  0.55654796,  0.54728111,  0.54550425,  0.54442239]),
 'mean_test_score': array([-0.45661863, -0.45661903, -0.45662441, -0.45668707, -0.45720272,
        -0.45810941, -0.45915906, -0.46043816, -0.4595911 , -0.46073245]),
 'mean_train_score': array([-0.45503394, -0.45503394, -0.45503393, -0.45503393, -0.45503393,
        -0.45503393, -0.45503393, -0.45503393, -0.45503393, -0.45503393]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga'],
              mask =

In [16]:
print(-grid.best_score_)
print(grid.best_params_)

0.456618630337
{'solver': 'sag', 'C': 0.01}


In [42]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 
cost time:57
Train Accuary: 83.00%
Train log_loss:  0.455267064789


['../data/project_2/models/Onehot_A/LR_C0.01_A_hour.sklearn']

# A_xgb

In [5]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat_xgb')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [6]:
#导入数据

print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [7]:
X_train.shape, y_train.shape

((4042898, 51200), (4042898,))

In [8]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss', n_jobs=4)


In [ ]:
grid.fit(X_train,y_train)
grid.cv_results_

In [ ]:
print(-grid.best_score_)
print(grid.best_params_)

In [ ]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 
cost time:32
Train Accuary: 83.00%
Train log_loss:  0.453926167587


# B_xgb

In [ ]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat_xgb')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [ ]:
#导入数据

print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

In [ ]:
X_train.shape, y_train.shape

In [ ]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')


In [ ]:
grid.fit(X_train,y_train)
grid.cv_results_

In [ ]:
print(-grid.best_score_)
print(grid.best_params_)

In [ ]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 
cost time:32
Train Accuary: 83.00%
Train log_loss:  0.453926167587


# B_cat

In [39]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [23]:
#导入数据

print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [24]:
X_train.shape, y_train.shape

((4042898, 901808), (4042898,))

In [25]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')


In [26]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached w

{'mean_fit_time': array([  40.3663396 ,   49.49943328,   49.84781265,   80.88739292,
         199.45493261,  206.05165696,  197.66245071,  202.43555601,
         196.41139174,  202.96178778]),
 'mean_score_time': array([ 0.52589401,  0.52809691,  0.52170324,  0.52200667,  0.52575588,
         0.52752177,  0.52544602,  0.52752463,  0.52813784,  0.52762826]),
 'mean_test_score': array([-0.45512197, -0.45512413, -0.45423192, -0.45423608, -0.45474148,
        -0.45464744, -0.4651223 , -0.45945798, -0.47023533, -0.46088521]),
 'mean_train_score': array([-0.45434805, -0.45434948, -0.44931142, -0.44931736, -0.42797456,
        -0.42882026, -0.40228668, -0.41147112, -0.39782552, -0.40888018]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga'],
              mask =

In [27]:
print(-grid.best_score_)
print(grid.best_params_)

0.454231915964
{'solver': 'sag', 'C': 0.1}


In [38]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.1

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 
cost time:124
Train Accuary: 83.02%
Train log_loss:  0.449215597138


['../data/project_2/models/Onehot_B/LR_C0.1_B_cat.sklearn']

# A_his

In [28]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_his')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [29]:
#导入数据

print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [30]:
X_train.shape, y_train.shape

((4042898, 30), (4042898,))

In [31]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')


In [32]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site

{'mean_fit_time': array([ 20.17629027,  28.43019478,  26.59142311,  26.69366582,
         40.89491367,  27.82930295,  56.68577949,  29.24172815,
         57.9981335 ,  28.83819652]),
 'mean_score_time': array([ 0.52473609,  0.5248847 ,  0.52490401,  0.52642624,  0.52567863,
         0.53004837,  0.52974876,  0.53003057,  0.52338767,  0.52351912]),
 'mean_test_score': array([-0.45439099, -0.45439104, -0.45443027, -0.45443022, -0.4544359 ,
        -0.45443588, -0.45443643, -0.45443644, -0.45443649, -0.45443651]),
 'mean_train_score': array([-0.45384646, -0.45384646, -0.45383968, -0.45383968, -0.4538396 ,
        -0.4538396 , -0.4538396 , -0.4538396 , -0.4538396 , -0.4538396 ]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga'],
              mask = [False Fa

In [33]:
print(-grid.best_score_)
print(grid.best_params_)

0.454390991972
{'solver': 'sag', 'C': 0.01}


In [35]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


Load Data
training . . . 
cost time:32
Train Accuary: 83.00%
Train log_loss:  0.453926167587


['../data/project_2/models/Onehot_A/LR_C0.01_A_his.sklearn']

# 结论

In [5]:
X_train, y_train = MergeNpz(output_path, file_name, data_begins, threshold,)
y_train = y_train.toarray().astype(np.float32)[0]

c=

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


total shape:  (4042898, 645182)  |  (1, 4042898)


In [8]:
#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=1, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

training . . . 


/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


cost time:549
Train Accuary: 83.77%
Train log_loss:  0.3737715081


# 'C': 0.1

In [25]:
c=0.1
#保存模型
joblib.dump(lr, model_path+'LR_C{0}.sklearn_more{1}__'.format(c,threshold))


['../data/project_2/models/Onehot_A/LR_C0.1.sklearn_more10__']

In [ ]:
lr_model = joblib.load(model_path+model_path+'LR_C{0}.sklearn'.format(c))